# 缺失值主要有三种形式：null、NaN 或 NA

# 1-选择处理缺失值的方法

识别缺失值的方法：
    
    一种方法是通过一个覆盖全局的掩码表示缺失值，
    另一种方法是用一个标签值（sentinel value）表示缺失值。
    
    在掩码方法中：
        掩码可能是一个与原数组维度相同的完整布尔类型数组，
        也可能是用一个比特（0 或 1）表示有缺失值的局部状态。
    
    在标签方法中：
        标签值可能是具体的数据（例如用 -9999 表示缺失的整数），
        也可能是些极少出现的形式。
        另外，标签值还可能是更全局的值，比如用 NaN（不是一个数）表示缺失的浮点数，
        它是 IEEE 浮点数规范中指定的特殊字符。

# 2-Pandas 的缺失值

01-None: Python 对象类型的缺失值
    
    Pandas 可以使用的第一种缺失值标签是 None，它是一个 Python 单体对象，经常在代码中表示缺失值.
    由于 None 是一个 Python 对象，所以不能作为任何 NumPy / Pandas 数组类型的缺失值，只能用于 'object' 数组类型.

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

这里 dtype=object 表示 NumPy 认为由于这个数组是 Python 对象 构成的，因此将其类型判断为 object。（比其他原生类型数组要消耗更多的资源）

In [3]:
for dtype in ['object', 'int']:
    print('dtype=', dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype= object
59.2 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype= int
2.33 ms ± 12 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



使用 Python 对象构成的数组就意味着如果你对一个包含 None 的数组进行累计操作，如 sum() 或者 min()，那么通常会出现类型错误.

In [4]:
# 在 Python 中没有定义整数与 None 之间的加法运算
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

02-NaN：数值类型的缺失值
    
    另一种缺失值的标签是 NaN（全称 Not a Number，不是一个数），
    是一种按照 IEEE 浮点数标准设计、在任何系统中都兼容的 特殊浮点数

In [5]:
vals2 = np.array([1, np.nan,3,4])
vals2.dtype

dtype('float64')

请注意，NumPy 会为这个数组选择一个原生浮点类型，这意味着和之前的 object 类型数组不同，这个数组会被编译成 C 代码从而实现快速操作。可以把 NaN 看作是一个数据类病毒：它会将与它接触过的数据同化.

In [6]:
# 无论和 NaN 进行何种操作，最终结果都是 NaN
1 + np.nan

nan

In [7]:
0 * np.nan

nan

In [8]:
# 虽然这些累计操作的结果定义是合理的（即不会抛出异常），但是并非总是有效的
vals2.sum(),vals2.min(),vals2.max()

(nan, nan, nan)

In [9]:
# NumPy 也提供了一些特殊的累计函数，它们可以忽略缺失值的影响：
np.nansum(vals2),np.nanmin(vals2),np.nanmax(vals2)

(8.0, 1.0, 4.0)

谨记，NaN 是一种特殊的浮点数，不是整数、字符串以及其他数据类型

03-Pands 中的 NaN 与 None 的差异

In [10]:
# 虽然 NaN 与 None 各有各的用处，但是 Pandas 把它们看成是可以等价交换的
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [11]:
# Pandas 会将没有标签值的数据类型自动转换为 NA
# 当将整型数组中的一个值设置为 np.nan 时，这个值就会强制转换成浮点数缺失值 NA

x = pd.Series(range(2),dtype=int)
x

0    0
1    1
dtype: int32

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

注意：除了将整型数组的缺失值强制转换为浮点数，Pandas 还会 自动将 None 转换为 NaN。

Pandas对不同类型缺失值的转换规则
    
    类型                缺失值转换规则        NA标签值
    floating 浮点型     无变化                np.nan 
    object 对象类型     无变化                None 或 np.nan 
    integer 整数类型    强制转换为            float64 np.nan 
    boolean 布尔类型    强制转换为            object None 或 np.nan

Pandas 中字符串类型的数据通常是用 object 类型存储的.

# 3-处理缺失值
    isnull()--创建一个布尔类型的掩码标签缺失值。
    notnull()--与 isnull() 操作相反。
    dropna()--返回一个剔除缺失值的数据。
    fillna()--返回一个填充了缺失值的数据副本。

01--发现缺失值
    
    isnull() 和 notnull()
    都返回布尔类型的掩码数据

In [13]:
data = pd.Series([1,np.nan,'hello',None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [14]:
# 布尔类型掩码数组可以直接作为 Series 或 DataFrame 的索引使用
data[data.notnull()]

0        1
2    hello
dtype: object

isnull() 和 notnull() 同样适用于 DataFrame，产生的结果同样是布尔类型

02--剔除缺失值

    dropna()（剔除缺失值）和 fillna()（填充缺失值）

In [15]:
data.dropna()

0        1
2    hello
dtype: object

In [16]:
# 在 DataFrame 上使用它们时需要设置一些参数
df = pd.DataFrame([[1,np.nan,2],
                  [2,3,5],
                  [np.nan,4,6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


从 DataFrame 中单独剔除一个值，要么是剔除缺失值所 在的整行，要么是整列

In [17]:
# 默认情况下，dropna() 会剔除任何包含缺失值的整行数据
df.dropna()

,0,1,2
1,2.0,3.0,5


In [18]:
# 设置按不同的坐标轴剔除缺失值，
# 比如 axis=1（或 axis='columns'）会剔除任何包含缺失值的整列数据.
df.dropna(axis='columns')

,2
0,2
1,5
2,6


但是这么做也会把非缺失值一并剔除，因为可能有时候只需要剔除 全部是缺失值的行或列，或者绝大多数是缺失值的行或列.

设置 how 或 thresh 参数来满足，它们可以设置剔除行 或列缺失值的数量阈值.

In [19]:
# 默认设置是 how='any'，也就是说只要有缺失值就剔除整行或整列
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [20]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [21]:
# 可以通过 thresh 参数设置行或列中非缺失值的最小数量
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


03-填充缺失值
    
    替换成有效的数值：
        1-可以通过isnull() 方法建立掩码来填充缺失值
        2- Pandas 为此专门提供 了一个 fillna() 方法，返回填充了缺失值后的数组副本

In [22]:
data = pd.Series([1,np.nan,3,None,3],index=list('abcde'))
data

a    1.0
b    NaN
c    3.0
d    NaN
e    3.0
dtype: float64

In [23]:
# 用一个 0 来填充缺失值
data.fillna(0)

a    1.0
b    0.0
c    3.0
d    0.0
e    3.0
dtype: float64

In [24]:
# 用缺失值前面的有效值来从前往后填充
data.fillna(method='ffill')

a    1.0
b    1.0
c    3.0
d    3.0
e    3.0
dtype: float64

In [25]:
# 用缺失值后面的有效值来从后往前填充
data.fillna(method='bfill')

a    1.0
b    3.0
c    3.0
d    3.0
e    3.0
dtype: float64

DataFrame 的操作方法与 Series 类似，只是在填充时需要设置坐 标轴参数 axis

In [26]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [27]:
df.fillna(method='ffill',axis=1) # 行

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [28]:
df.fillna(method='bfill',axis=1)

,0,1,2,3
0,1.0,2.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,4.0,4.0,6.0,NaN


In [29]:
df.fillna(method='ffill',axis=0) #列

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


In [30]:
df.fillna(method='bfill',axis=0)

,0,1,2,3
0,1.0,3.0,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


在从前往后填充时，需要填充的缺失值前面没 有值，那么它就仍然是缺失值。